In [29]:
import numpy as np
import tensorflow as tf
import cv2
import os
import glob
import random
from sklearn.model_selection import train_test_split
import tensorflow.keras.backend as K
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, Flatten, Dense, Dropout, Lambda
from tensorflow.keras.models import Model


In [30]:
IMG_SIZE = (128, 128)

# Function to load images from a folder
def load_images(folder):
    images = []
    for img_path in glob.glob(os.path.join(folder, "*.png")):
        img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
        if img is None:
            continue
        img = cv2.resize(img, IMG_SIZE) / 255.0  # Resize and normalize
        images.append(img)
    return np.array(images)

# Load datasets
original_images = load_images("signatures/full_org")
forged_images = load_images("signatures/full_forg")

# Reshape for CNN input
original_images = original_images.reshape(original_images.shape[0], 128, 128, 1)
forged_images = forged_images.reshape(forged_images.shape[0], 128, 128, 1)

print(f"Loaded {original_images.shape[0]} original and {forged_images.shape[0]} forged signatures.")



Loaded 1320 original and 1320 forged signatures.


In [31]:
def create_pairs(original, forged):
    pairs = []
    labels = []
    
    # Create positive pairs (Same person)
    for i in range(len(original) - 1):
        pairs.append([original[i], original[i+1]])
        labels.append(1)  # Same signature
    
    # Create negative pairs (Forgery)
    for i in range(len(original)):
        pairs.append([original[i], random.choice(forged)])
        labels.append(0)  # Different signature
    
    return np.array(pairs), np.array(labels)

# Generate training pairs
pairs, labels = create_pairs(original_images, forged_images)

# Shuffle and split data
from sklearn.utils import shuffle
pairs, labels = shuffle(pairs, labels, random_state=42)
X_train, X_test, y_train, y_test = train_test_split(pairs, labels, test_size=0.2, random_state=42)

print(f"Training Pairs: {X_train.shape[0]}, Testing Pairs: {X_test.shape[0]}")


Training Pairs: 2111, Testing Pairs: 528


In [33]:
# Define CNN feature extractor
def build_base_network(input_shape):
    model = tf.keras.Sequential([
        Conv2D(64, (3,3), activation='relu', input_shape=input_shape),
        MaxPooling2D(pool_size=(2,2)),
        
        Conv2D(128, (3,3), activation='relu'),
        MaxPooling2D(pool_size=(2,2)),
        
        Conv2D(256, (3,3), activation='relu'),
        MaxPooling2D(pool_size=(2,2)),
        
        Flatten(),
        Dense(256, activation='relu'),
        Dropout(0.5),
        Dense(128, activation='relu')
    ])
    return model

# Input layers
input_shape = (128, 128, 1)
input_A = Input(shape=input_shape)
input_B = Input(shape=input_shape)

# Shared CNN feature extractor
base_network = build_base_network(input_shape)

# Generate feature vectors
feat_A = base_network(input_A)
feat_B = base_network(input_B)

# Lambda layer to compute Euclidean distance
def euclidean_distance(vectors):
    x, y = vectors
    return K.sqrt(K.sum(K.square(x - y), axis=1, keepdims=True))

distance = Lambda(euclidean_distance)([feat_A, feat_B])

# Fully connected layer for classification
output = Dense(1, activation="sigmoid")(distance)

# Create Siamese model
siamese_model = Model(inputs=[input_A, input_B], outputs=output)

# Compile model
siamese_model.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

# Train model
history = siamese_model.fit(
    [X_train[:, 0], X_train[:, 1]], y_train, 
    epochs=10, batch_size=32, validation_data=([X_test[:, 0], X_test[:, 1]], y_test)
)


Epoch 1/10
66/66 ━━━━━━━━━━━━━━━━━━━━ 33s 478ms/step - accuracy: 0.4948 - loss: 0.6963 - val_accuracy: 0.5095 - val_loss: 0.6933
Epoch 2/10
66/66 ━━━━━━━━━━━━━━━━━━━━ 32s 483ms/step - accuracy: 0.4701 - loss: 0.6944 - val_accuracy: 0.5095 - val_loss: 0.6932
Epoch 3/10
66/66 ━━━━━━━━━━━━━━━━━━━━ 31s 472ms/step - accuracy: 0.5083 - loss: 0.6931 - val_accuracy: 0.5095 - val_loss: 0.6931
Epoch 4/10
66/66 ━━━━━━━━━━━━━━━━━━━━ 66s 1s/step - accuracy: 0.5004 - loss: 0.6932 - val_accuracy: 0.5095 - val_loss: 0.6931
Epoch 5/10
66/66 ━━━━━━━━━━━━━━━━━━━━ 31s 457ms/step - accuracy: 0.4879 - loss: 0.6933 - val_accuracy: 0.5095 - val_loss: 0.6931
Epoch 6/10
66/66 ━━━━━━━━━━━━━━━━━━━━ 32s 483ms/step - accuracy: 0.4996 - loss: 0.6932 - val_accuracy: 0.5095 - val_loss: 0.6931
Epoch 7/10
66/66 ━━━━━━━━━━━━━━━━━━━━ 34s 510ms/step - accuracy: 0.5073 - loss: 0.6931 - val_accuracy: 0.5095 - val_loss: 0.6931
Epoch 8/10
66/66 ━━━━━━━━━━━━━━━━━━━━ 35s 537ms/step - accuracy: 0.4801 - loss: 0.6932 - val_accurac

In [35]:
# Evaluate model on test data
test_loss, test_acc = siamese_model.evaluate([X_test[:, 0], X_test[:, 1]], y_test)
print(f"Test Accuracy: {test_acc:.2f}")


17/17 ━━━━━━━━━━━━━━━━━━━━ 2s 128ms/step - accuracy: 0.5174 - loss: 0.6931
Test Accuracy: 0.49


In [37]:
def predict_signature(img1_path, img2_path):
    # Load and preprocess both images
    img1 = cv2.imread(img1_path, cv2.IMREAD_GRAYSCALE)
    img2 = cv2.imread(img2_path, cv2.IMREAD_GRAYSCALE)

    if img1 is None or img2 is None:
        print("Error loading images. Check the file paths.")
        return

    # Resize and normalize images
    img1 = cv2.resize(img1, IMG_SIZE) / 255.0
    img2 = cv2.resize(img2, IMG_SIZE) / 255.0

    # Reshape for model input
    img1 = img1.reshape(1, 128, 128, 1)
    img2 = img2.reshape(1, 128, 128, 1)

    # Predict similarity
    similarity_score = siamese_model.predict([img1, img2])[0][0]

    # Define threshold
    threshold = 0.4  
    result = "Same Person (Match)" if similarity_score < threshold else "Different Person (Forgery)"

    print(f"Similarity Score: {similarity_score:.4f}")
    print(f"Prediction: {result}")

# Example usage: Compare two new signatures
predict_signature("original.png", "test.png")


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 57ms/step
Similarity Score: 0.5006
Prediction: Different Person (Forgery)


In [38]:
# Save the trained Siamese model
siamese_model.save("siamese_model.h5")
print("Model saved successfully!")


Model saved successfully!
